In [5]:
import boto3
import sagemaker
import json

with open('credentials.json') as f:
    credentials = json.load(f)

ecr_namespace = "sagemaker-training-containers/"
prefix = "basic-training-container"
role = credentials['role']
account_id = credentials['account_id']

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()
ecr_repository_name = ecr_namespace + prefix

In [6]:
sagemaker_session

In [2]:
!#sudo docker build -f ./../docker/Dockerfile -t sagemaker-training-containers/basic-training-container ../docker

In [4]:
%%capture
! bash ./scripts/build_and_push.sh $account_id $region $ecr_repository_name

In [11]:
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(
    account_id, region, ecr_repository_name
)
print(container_image_uri)

855373717346.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-training-containers/basic-training-container:latest


In [8]:
! echo "val1, val2, val3" > dummy.csv
print(sagemaker_session.upload_data("dummy.csv", bucket, prefix + "/train"))
print(sagemaker_session.upload_data("dummy.csv", bucket, prefix + "/val"))
! rm dummy.csv

s3://sagemaker-eu-central-1-855373717346/basic-training-container/train/dummy.csv
s3://sagemaker-eu-central-1-855373717346/basic-training-container/val/dummy.csv


In [9]:
import sagemaker

# Instances Pricing https://aws.amazon.com/sagemaker/pricing/
est = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    train_instance_count=1,
    #train_instance_type="local",  # use local mode
    train_instance_type='ml.m5.large',
    base_job_name=prefix,
)

est.set_hyperparameters(hp1="value1", hp2=300, hp3=0.001)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [10]:
train_config = sagemaker.session.s3_input(
    "s3://{0}/{1}/train/".format(bucket, prefix), content_type="text/csv"
)
val_config = sagemaker.session.s3_input(
    "s3://{0}/{1}/val/".format(bucket, prefix), content_type="text/csv"
)

est.fit({"train": train_config, "validation": val_config})

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-08-05 18:36:59 Starting - Starting the training job...
2022-08-05 18:37:24 Starting - Preparing the instances for trainingProfilerReport-1659724618: InProgress
...
2022-08-05 18:38:18 Downloading - Downloading input data...
2022-08-05 18:39:05 Training - Training image download completed. Training in progress.Running training...
Hyperparameters configuration:
{'hp1': 'value1', 'hp2': '300', 'hp3': '0.001'}
Input data configuration:
{'train': {'ContentType': 'text/csv',
           'RecordWrapperType': 'None',
           'S3DistributionType': 'FullyReplicated',
           'TrainingInputMode': 'File'},
 'validation': {'ContentType': 'text/csv',
                'RecordWrapperType': 'None',
                'S3DistributionType': 'FullyReplicated',
                'TrainingInputMode': 'File'}}
List of files in train channel: 
/opt/ml/input/data/train/dummy.csv
List of files in validation channel: 
/opt/ml/input/data/validation/dummy.csv
Resource configuration:
{'current_group_name': 'hom